In [ ]:
# 부스팅 알고리즘 : 여러 개의 약한 학습기(weak learner)를 순차적으로 학습-예측하면서
# 잘못 예측한 데이터에 가중치 부여를 통해 오류를 개선해 나가면서 학습하는 방식

In [ ]:
# 경사 하강법(Gradient Descent)
# - 오류식 h(x) = y - F(x)을 최소화하는 방향성을 가지고
#   반복적으로 가중치 값을 업데이터 하는 것
# 즉, 반복 수행을 통해 오류를 최소화할 수 있도록 가중치의 업데이트 값을 도출하는 기법

In [ ]:
# 사이킷런은 GBM 기반의 분류를 위해서 
# GradientBoostingClassifer 클래스를 제공

In [ ]:
# 사이킷런의 GBM을 이용해 사용자 행동 데이터 세트 예측/분류
# GBM으로 학습하는 시간이 얼마나 걸리는지 수행 시간도 같이 측정

In [1]:
from sklearn.ensemble import GradientBoostingClassifier
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_human_dataset() :
    
    # 각 데이터 파일들은 공백으로 분리되어 있으므로
    # read_csv에서 공백 문자를 sep로 할당
    feature_name_df = pd.read_csv('./data/human_activity/features.txt', sep='\s+',
                                 header=None, names=['column_index', 'column_name'])
    # DataFrame에 피처명을 칼럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = feature_name_df.iloc[:, 1].values.tolist()
    
    # 학습 피처 데이터 세트와 테스트 피처 데이터를 DataFrame으로 로딩
    # 칼럼명은 feature_name 적용
    X_train = pd.read_csv('./data/human_activity/train/X_train.txt', sep = '\s+', names = feature_name)
    X_test = pd.read_csv('./data/human_activity/test/X_test.txt', sep = '\s+', names = feature_name )
    
    # 학습 레이블과 테스트 레이블 데이터를 DataFrame으로 로딩
    # 칼럼명은 action으로 부여
    y_train = pd.read_csv('./data/human_activity/train/y_train.txt', sep = '\s+', header = None, names = ['action'])
    y_test = pd.read_csv('./data/human_activity/test/y_test.txt', sep = '\s+', header = None, names = ['action'])
    
    # 로드된 학습/테스트용 DataFrame 모두 반환
    return X_train, X_test, y_train, y_test

In [3]:
X_train, X_test, y_train, y_test = get_human_dataset()

In [4]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7352, 561), (2947, 561), (7352, 1), (2947, 1))

In [6]:
# GBM 수행 시간 측정을 위함
# 시작 시간 설정
start_time = time.time()

In [8]:
from sklearn.metrics import accuracy_score

In [9]:
gb_clf = GradientBoostingClassifier(random_state = 0)
gb_clf.fit(X_train, y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)

print('GBM 정확도 : {0:.4f}'.format(gb_accuracy))
print('GBM 수행 시간 : {0:.1f}초'.format(time.time() - start_time))

GBM 정확도 : 0.9376
GBM 수행 시간 : 1140.5초


#### GBM 하이퍼 파라미터 튜닝

In [11]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators' : [100, 500],
    'learning_rate' : [0.05, 0.1]
}

grid_cv = GridSearchCV(gb_clf, param_grid = params, cv=2, verbose = 1)
grid_cv.fit(X_train, y_train)
print('최적 하이퍼 파라미터 : \n', grid_cv.best_params_)
print('최고 예측 정확도 : {0:.4f}'.format(grid_cv.best_score_))

Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed: 68.6min finished


최적 하이퍼 파라미터 : 
 {'learning_rate': 0.05, 'n_estimators': 500}
최고 예측 정확도 : 0.9010


In [12]:
# GridSearchCV를 이용해 최적으로 학습된 estimator로 예측 수행
gb_pred = grid_cv.best_estimator_.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
print('GBM 정확도 : {0:.4f}'.format(gb_accuracy))

GBM 정확도 : 0.9410
